In [ ]:
## import packages
from datetime import datetime
import os
import pandas as pd
import numpy as np
import random
from itertools import chain
import hail as hl
from hail.linalg import BlockMatrix

In [2]:
DATASET = os.getenv('WORKSPACE_CDR')
bucket = os.getenv('WORKSPACE_BUCKET')

## QC

1. Liftover, key the sumstats by locus  
2. Restrict to biallelic variants  
3. Negative strand problems  
4. Annotate to matrix table, and swap ref:alt if necessary  

## Liftover, then key the sumstats by locus (done in Step2a)

rg37 = hl.get_reference('GRCh37')  
rg38 = hl.get_reference('GRCh38')   
rg37.add_liftover('gs://hail-common/references/grch37_to_grch38.over.chain.gz', rg38)  

In [4]:
# biallelic SNP after AFR-specific QC
var_wgs_afr = hl.read_table(f'{bucket}/WGSData/WGS_Vars_Ancestry_AFR_QCed.ht')
var_wgs_afr = var_wgs_afr.key_by("locus")

Initializing Hail with default parameters...
/opt/conda/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.0
SparkUI available at http://all-of-us-11150-m.us-central1-c.c.terra-vpc-sc-fd39b54c.internal:41389
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130.post1-c69cd67afb8b
LOGGING: writing to /home/jupyter/workspaces/prswithwgsvsarraydata/hail-20241117-2009-0.2.130.post1-c69cd67afb8b.log


### Quantatative traits

Use `beta_meta_hq`, `se_meta_hq`, `neglog10_pval_meta_hq`

Add `var_wgs = var_wgs.repartition(2000)`, var_wgs = var_wgs.checkpoint(ht_filename_check`, overwrite=True`)

In [7]:
# Step2a_Array_SumStats_QC.ipynb performed some preliminary QC for sumstats
# The intermediate file is saved as checkpoint
# start from the checkpoint files

def sumstats_QC_quant(ht_filename_in, ht_filename_check, filename_out, var_wgs):
    sumstats = hl.read_table(ht_filename_in)
    
    ##############################################################################################################
    ###################################### Analysis done in Step2a_Array_SumStats_QC.ipynb #######################
    # most of the fields are irrelevant for our analysis, drop it first to save computation                      #
    # biallelic SNPs                                                                                             #
    # check if is_SNP                                                                                            #                                                                                   #
    # flip negative strand                                                                                       #
    ##############################################################################################################
    ##############################################################################################################
    
    var_wgs = var_wgs.repartition(2000) ###***
    
    ############## check ref:alt, flip if necessary ############
    var_wgs = var_wgs.annotate(**sumstats[var_wgs.locus])
    var_wgs = var_wgs.filter((~hl.is_nan(var_wgs.beta_meta_hq)) &
                                  (~hl.is_nan(var_wgs.se_meta_hq)) &
                                  (~hl.is_nan(var_wgs.neglog10_pval_meta_hq)))
    
    var_wgs = var_wgs.annotate(beta_meta_fix_ref_alt = hl.case()
        .when(((var_wgs.alleles[0] == var_wgs.alleles_fix_neg_strand[0]) & (var_wgs.alleles[1] == var_wgs.alleles_fix_neg_strand[1])), var_wgs.beta_meta_hq) 
        .when(((var_wgs.alleles[0] == var_wgs.alleles_fix_neg_strand[1]) & (var_wgs.alleles[1] == var_wgs.alleles_fix_neg_strand[0])), -var_wgs.beta_meta_hq)
        .default(float('nan')))
    
    var_wgs = var_wgs.checkpoint(ht_filename_check, overwrite=True)
    
    sumstats_QCed = var_wgs.select(
                        rsid = var_wgs.rsid,
                        alleles1_wgs = var_wgs.alleles[0],
                        alleles2_wgs = var_wgs.alleles[1],
                        alleles1_sumstats_original = var_wgs.ref,
                        alleles2_sumstats_original = var_wgs.alt,
                        is_negative_strand = var_wgs.is_negative_strand,
                        alleles1_sumstats_fixstrand = var_wgs.alleles_fix_neg_strand[0],
                        alleles2_sumstats_fixstrand = var_wgs.alleles_fix_neg_strand[1],
                        beta_meta = var_wgs.beta_meta_hq,
                        beta_meta_fix_ref_alt = var_wgs.beta_meta_fix_ref_alt,
                        se_meta = var_wgs.se_meta_hq,
                        neglog10_pval_meta = var_wgs.neglog10_pval_meta_hq)
    
    sumstats_QCed.export(filename_out)

In [8]:
print("HDL_AFR " + str(datetime.now()))
sumstats_QC_quant(f"{bucket}/Sumstats/biomarkers-30760-both_sexes-irnt_checkpoint2.ht",
                  f"{bucket}/Sumstats/biomarkers-30760-both_sexes-irnt_checkpoint3.ht",
                  f"{bucket}/Sumstats/WGS_HDL_afr_QCed.tsv.bgz",
                  var_wgs_afr)

HDL_AFR 2024-11-17 20:10:40.042188


2024-11-17 20:14:43.969 Hail: INFO: wrote table with 7621041 rows in 2000 partitions to gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/biomarkers-30760-both_sexes-irnt_checkpoint3.ht
2024-11-17 20:14:54.251 Hail: INFO: merging 2001 files totalling 127.0M... 2000]
2024-11-17 20:14:59.672 Hail: INFO: while writing:
    gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/WGS_HDL_afr_QCed.tsv.bgz
  merge time: 5.420s
